# Flight Delay prediction

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn,optim
import torch.nn.functional as F
import datetime
from torch.utils import data
from random import sample,seed

## Importing Flight Data

In [2]:
flights = pd.read_csv("./data/flights.csv",low_memory=False)
airlines = pd.read_csv("./data/airlines.csv")
airports = pd.read_csv("./data/airports.csv")

## Preprocessing of the data

In [3]:
airports = {ch:i for i,ch in enumerate(flights.ORIGIN_AIRPORT.unique())}
airlines = {ch:i for i,ch in enumerate(airlines.IATA_CODE)}
airports['10666'] =len(airports)

In [4]:
# one hot encoder for day of the week
def one_hot_encode(size,val):
    a = np.zeros((size,),dtype=int)
    a[(val-1)] = 1
    return a

In [5]:
# Date and time preprocess copied from
# link - https://www.kaggle.com/fabiendaniel/predicting-flight-delays-tutorial/data
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
# def format_heure(chaine):
#     if pd.isnull(chaine):
#         return np.nan
#     else:
#         if chaine == 2400: chaine = 0
#         chaine = "{0:04d}".format(int(chaine))
#         heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
#         return heure

In [6]:
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH', 'DAY']])

In [7]:
# REFORMATING THE DATE VARIABLES
#__________________________________________________________________________________
# flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_heure) #create_flight_time(flights, 'SCHEDULED_DEPARTURE')
# flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_heure)
# flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_heure)
# flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_heure)
#__________________________________________________________________________
flights.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
             'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY
0,5,430,2354.0,408.0,-11.0,-22.0
1,10,750,2.0,741.0,-8.0,-9.0
2,20,806,18.0,811.0,-2.0,5.0
3,20,805,15.0,756.0,-5.0,-9.0
4,25,320,24.0,259.0,-1.0,-21.0
5,25,602,20.0,610.0,-5.0,8.0


In [8]:
# DROPING IRRELEVANT COLUMNS
#_____________________________________________________________________________________
variables_to_remove = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF', 'YEAR', 
                       'DATE', 'AIR_SYSTEM_DELAY',
                       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                       'WEATHER_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                       'FLIGHT_NUMBER', 'TAIL_NUMBER', 'AIR_TIME']
flights.drop(variables_to_remove, axis = 1, inplace = True)
flights = flights[['AIRLINE','DAY', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
        'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
        'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
        'SCHEDULED_TIME', 'ELAPSED_TIME','MONTH','DAY_OF_WEEK']]
flights[:5]

,AIRLINE,DAY,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,SCHEDULED_TIME,ELAPSED_TIME,MONTH,DAY_OF_WEEK
0,AS,1,ANC,SEA,5,2354.0,-11.0,430,408.0,-22.0,205.0,194.0,1,4
1,AA,1,LAX,PBI,10,2.0,-8.0,750,741.0,-9.0,280.0,279.0,1,4
2,US,1,SFO,CLT,20,18.0,-2.0,806,811.0,5.0,286.0,293.0,1,4
3,AA,1,LAX,MIA,20,15.0,-5.0,805,756.0,-9.0,285.0,281.0,1,4
4,AS,1,SEA,ANC,25,24.0,-1.0,320,259.0,-21.0,235.0,215.0,1,4


In [9]:
# PROCESSING THE CATEGORICAL DATA
#___________________________________________________________________
flights['AIRLINE'] = flights['AIRLINE'].apply(lambda x: airlines[x])
flights['ORGIN_AIRPORT_V'] = flights['ORIGIN_AIRPORT'].apply(lambda x: airports[x])
flights['DESTINATION__AIRPORT_V'] = flights['DESTINATION_AIRPORT'].apply(lambda x: airports[x])
flights['DAY_OF_WEEK'] = flights['DAY_OF_WEEK'].apply(lambda x: one_hot_encode(7,x))
flights['DAY'] = flights['DAY'].apply(lambda x: one_hot_encode(31,x))
flights['MONTH'] = flights['MONTH'].apply(lambda x: one_hot_encode(12,x))
flights = flights[['AIRLINE','ORGIN_AIRPORT_V',"DESTINATION__AIRPORT_V",'ARRIVAL_DELAY','DAY_OF_WEEK','DEPARTURE_DELAY','DAY','MONTH','SCHEDULED_ARRIVAL']]
flights.head()

,AIRLINE,ORGIN_AIRPORT_V,DESTINATION__AIRPORT_V,ARRIVAL_DELAY,DAY_OF_WEEK,DEPARTURE_DELAY,DAY,MONTH,SCHEDULED_ARRIVAL
0,6,0,3,-22.0,"[0, 0, 0, 1, 0, 0, 0]",-11.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",430
1,1,1,77,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-8.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",750
2,2,2,97,5.0,"[0, 0, 0, 1, 0, 0, 0]",-2.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",806
3,1,1,84,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-5.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",805
4,6,3,0,-21.0,"[0, 0, 0, 1, 0, 0, 0]",-1.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",320


In [10]:
# CLEAN UP
#______________________________
flights.dropna(inplace =True)
flights = flights.reset_index()
flights = flights[['AIRLINE','ORGIN_AIRPORT_V',"DESTINATION__AIRPORT_V",'ARRIVAL_DELAY','DAY_OF_WEEK','DEPARTURE_DELAY','DAY','MONTH','SCHEDULED_ARRIVAL']]
flights.head()

,AIRLINE,ORGIN_AIRPORT_V,DESTINATION__AIRPORT_V,ARRIVAL_DELAY,DAY_OF_WEEK,DEPARTURE_DELAY,DAY,MONTH,SCHEDULED_ARRIVAL
0,6,0,3,-22.0,"[0, 0, 0, 1, 0, 0, 0]",-11.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",430
1,1,1,77,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-8.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",750
2,2,2,97,5.0,"[0, 0, 0, 1, 0, 0, 0]",-2.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",806
3,1,1,84,-9.0,"[0, 0, 0, 1, 0, 0, 0]",-5.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",805
4,6,3,0,-21.0,"[0, 0, 0, 1, 0, 0, 0]",-1.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",320


In [11]:
flights.DAY

0          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                 ...                        
5714003    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5714004    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5714005    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5714006    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5714007    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: DAY, Length: 5714008, dtype: object

In [12]:
# MAKING THE SEQUENCES OF INPUT DATA
#________________________________________________________________
spequence_length = 1000
output_length = 1
start = 0
data_batch = {}
decoder_input ={}
labels = {}
for i,x in enumerate(range(0,len(flights),spequence_length)):
#     print('id_{}:{}'.format(i,x))
#     y = x+spequence_length
    data_batch['id_{}'.format(i)] = [i for i in range(start,x)]
    air = flights.loc[x:x+output_length-1,:]
    f = flights.loc[x:x+output_length-1,'ARRIVAL_DELAY']
    decoder_input['id_{}'.format(i)] = air
    labels['id_{}'.format(i)] = f.to_list()
    start = x

In [13]:
# DATA GENERATOR CLASS FOR EASY ACCESS AND PARALLELIZATION
#___________________________________________________________
class DataGen(data.Dataset):
    def __init__(self,list_id,labels):
        self.list_id = list_id
        self.labels = labels
    def __len__(self):
        return len(self.list_id)
    def __getitem__(self,index):
        ID = self.list_id[index]
        ind = data_batch[ID]
        X = flights.loc[ind,:]
        d_inp = decoder_input[ID]
        d_inp = {'DAY_OF_WEEK':torch.tensor(d_inp.DAY_OF_WEEK.to_list(),dtype=torch.float),
                'AIRLINE': torch.tensor(d_inp.AIRLINE.to_list(),dtype=torch.float),
                 'DAY':torch.tensor(d_inp.DAY.to_list(),dtype=torch.float),
                 'MONTH':torch.tensor(d_inp.MONTH.to_list(),dtype=torch.float),
                 'SCHEDULED_ARRIVAL':torch.tensor(d_inp.SCHEDULED_ARRIVAL.to_list(),dtype=torch.float),
                'ORGIN_AIRPORT_V': torch.tensor(d_inp.ORGIN_AIRPORT_V.to_list(),dtype=torch.float),
                  'DESTINATION__AIRPORT_V': torch.tensor(d_inp.DESTINATION__AIRPORT_V.to_list(),dtype=torch.float)}
        y = torch.tensor(labels[ID],dtype=torch.float)
        return d_inp,y,{'DAY_OF_WEEK':torch.tensor(X.DAY_OF_WEEK.to_list(),dtype=torch.float),
                  'AIRLINE': torch.tensor(X.AIRLINE.to_list(),dtype=torch.float),
                  'DAY':torch.tensor(X.DAY.to_list(),dtype=torch.float),
                 'MONTH':torch.tensor(X.MONTH.to_list(),dtype=torch.float),
                 'SCHEDULED_ARRIVAL':torch.tensor(X.SCHEDULED_ARRIVAL.to_list(),dtype=torch.float),
                  'ARRIVAL_DELAY': torch.tensor(X.ARRIVAL_DELAY.to_list(),dtype=torch.float),
                  'ORGIN_AIRPORT_V': torch.tensor(X.ORGIN_AIRPORT_V.to_list(),dtype=torch.float),
                  'DESTINATION__AIRPORT_V': torch.tensor(X.DESTINATION__AIRPORT_V.to_list(),dtype=torch.float)}

In [14]:
# PARAMETER FOR THE GENERATOR
#_____________________________
params = {'batch_size': 32,
          'shuffle': False,
          'num_workers': 6}

In [15]:
# SPLITING UP THE DATA INTO TRAINING AND VALIDATION
#_____________________________________________________
seed(1234)
data_ids=list(data_batch.keys())
data_ids.pop(0)
length = int(len(data_ids)*0.75)
training_ids = sample(data_ids,length)
validation_ids = []
for i in list(data_batch.keys()):
    if i not in training_ids:
        validation_ids.append(i)

In [16]:
#CREATING DATALOADERS
#_______________________________________________________________
training_set = DataGen(training_ids, labels)
training_generator = data.DataLoader(training_set, **params)
validation_ids.pop(0)
validation_set = DataGen(validation_ids, labels)
validation_generator = data.DataLoader(validation_set,batch_size=2,shuffle=False)

# MODEL DEFINITION

In [17]:
cuda =torch.cuda.is_available()
device = torch.device('cuda:0'if torch.cuda.is_available() else "cpu")
cuda

True

In [27]:
# MODEL DEFINITION
#_________________
class model(nn.Module):
    
    def __init__(self,airline_embed_size=4,airport_embed_size=8,hidden_size=83,no_of_layers=2,dropout=0.25,linear_size=1024):
        super(model,self).__init__()
        
        # DEFINING THE VARIABLES AND PARAMETERS
#         self.airline_embed_size = airline_embed_size
#         self.airport_embed_size = airport_embed_size
        self.hidden_size = hidden_size
        self.no_of_layers =no_of_layers
        self.hidden_size = (1+2+7+12+31+1) + 2048*(1+2+7+1+12+31+1) #(airport_embed_size+airline_embed_size+7) + 2048*(airport_embed_size+airline_embed_size+7+1)
        
        # MODEL ELEMENTS
#         self.airline_embed = nn.Embedding(len(airlines),airline_embed_size)
#         self.airport_embed = nn.Embedding(len(airports),airport_embed_size) 
        self.conv1 = nn.Conv1d(1000,2048,kernel_size=2,dilation=2,padding=1)
        self.conv2 = nn.Conv1d(2048,2048,kernel_size=2,dilation=2,padding=1)

        # HIDDEN SIZE IS CALCULATED BASED ON DOWN SIZING THROUGH CONVOLUTION
        self.bnorm = nn.BatchNorm1d(num_features=linear_size)
        self.linear_1 = nn.Linear(self.hidden_size,linear_size)
        self.linear_2 = nn.Linear(linear_size,linear_size)
        self.out = nn.Linear(linear_size,1)
        self.drop = nn.Dropout(p=dropout)
        
    def forward(self,x,decode):
#         print(x['AIRLINE'].is_cuda)
        decode['DAY_OF_WEEK'] = decode['DAY_OF_WEEK'].to(device)
        decode['SCHEDULED_ARRIVAL'] = decode['SCHEDULED_ARRIVAL'].to(device)
        decode['DAY'] = decode['DAY'].to(device)
        decode['MONTH'] = decode['MONTH'].to(device)
        decode['AIRLINE'] = decode['AIRLINE'].to(device)
        decode['DESTINATION__AIRPORT_V'] = decode['DESTINATION__AIRPORT_V'].to(device)
        decode['ORGIN_AIRPORT_V'] = decode['ORGIN_AIRPORT_V'].to(device)
        # CONDTIONALS MOVED TO DEVICE
        x['DAY_OF_WEEK'] = x['DAY_OF_WEEK'].to(device)
        x['SCHEDULED_ARRIVAL'] = x ['SCHEDULED_ARRIVAL'].to(device)
        x['SCHEDULED_ARRIVAL'] = x ['SCHEDULED_ARRIVAL'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        x['MONTH'] = x['MONTH'].to(device)
        x['DAY'] = x['DAY'].to(device)
        x['AIRLINE'] = x['AIRLINE'].to(device)
        x['ARRIVAL_DELAY'] = x['ARRIVAL_DELAY'].to(device)
        x['ARRIVAL_DELAY'] = x['ARRIVAL_DELAY'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        x['AIRLINE']=x['AIRLINE'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        x['DESTINATION__AIRPORT_V'] =  x['DESTINATION__AIRPORT_V'].to(device)
        x['DESTINATION__AIRPORT_V'] =x['DESTINATION__AIRPORT_V'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        x['ORGIN_AIRPORT_V'] = x['ORGIN_AIRPORT_V'].to(device)
        x['ORGIN_AIRPORT_V'] = x['ORGIN_AIRPORT_V'].reshape(x['DAY_OF_WEEK'].shape[0],x['DAY_OF_WEEK'].shape[1],-1)
        input_ =torch.cat((x['MONTH'],x['DAY'],x['SCHEDULED_ARRIVAL'],x['DAY_OF_WEEK'],x['AIRLINE'],x['DESTINATION__AIRPORT_V'],x['ORGIN_AIRPORT_V'],x['ARRIVAL_DELAY']),2)
        # CONCATINATED DAY OF THE WEEK, DELAY, AIRLINE AND DESTINATION-ORIGIN(GRAPH REP)
        
#         decode['AIRLINE'] = self.airline_embed(decode['AIRLINE'])
#         decode['DESTINATION__AIRPORT_V'] = self.airport_embed(decode['DESTINATION__AIRPORT_V'])
#         decode['ORGIN_AIRPORT_V'] = self.airport_embed(decode['ORGIN_AIRPORT_V'])

        out = F.relu(self.conv1(input_))
        out = F.relu(self.conv2(out))
        
        out = out.view(out.shape[0],-1)
        decode['SCHEDULED_ARRIVAL'] = decode['SCHEDULED_ARRIVAL'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['DAY'] = decode['DAY'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['MONTH'] = decode['MONTH'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['ORGIN_AIRPORT_V'] = decode['ORGIN_AIRPORT_V'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['AIRLINE'] = decode['AIRLINE'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['DESTINATION__AIRPORT_V'] = decode['DESTINATION__AIRPORT_V'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        decode['DAY_OF_WEEK'] = decode['DAY_OF_WEEK'].view(decode['ORGIN_AIRPORT_V'].shape[0],-1)
        out = torch.cat((out,decode['MONTH'],decode['DAY'],decode['SCHEDULED_ARRIVAL'],decode['DAY_OF_WEEK'],decode['DESTINATION__AIRPORT_V'],decode['ORGIN_AIRPORT_V'],decode['AIRLINE']),1)
        
        out = self.drop(F.relu(self.linear_1(out)))
        out = self.drop(F.relu(self.linear_2(out)))
        out = self.out(out)
        
        return out
    

In [28]:
#LOSS AND OPTIMISATION FUNCTIONS
conv = model()
criterian = nn.MSELoss() #nn.SmoothL1Loss() 
# It uses a squared term if the absolute error falls below 1 and an absolute term otherwise.
optimizer = optim.Adam(conv.parameters(),lr =0.001)

# Training

In [29]:
epoch = 25
conv.to(device)
g_loss = float("inf")
for e in range(0,epoch):
    running_loss =0
    conv.train()
    count =0
    for decode,y,x in training_generator: 
#         decode = decode.to(device)
        y = y.to(device)
        out = conv(x,decode)
        loss = criterian(out,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss +=loss.item()
        count =count+1
    else:
        if((e+1)%5==0):
            val_loss =0 
            conv.eval()
            count_val = 0
            for decode,y,x in validation_generator:
#                 decode = decode.to(device)
                y = y.to(device)
                out = conv(x,decode)
                loss = criterian(out,y)
                val_loss +=loss.item()
                count_val+=1
            print(f'The Training loss = {running_loss/count} : Validation loss is {val_loss/count_val}')
            if(val_loss < g_loss):
                print("Saving model....")
                torch.save(conv.state_dict(),"./conv.pt")
                g_loss = val_loss


The Training loss = 1678.4402363335907 : Validation loss is 1310.8454173984228
Saving model....
The Training loss = 2580.466380560576 : Validation loss is 1338.7563535239312
The Training loss = 1425.1797562784222 : Validation loss is 1325.325401498221
The Training loss = 995.06350503039 : Validation loss is 2973.4232463135377
The Training loss = 792.6598323992829 : Validation loss is 1721.4021620091025


# Testing

In [30]:
# spequence_length = 1000
conv.load_state_dict(torch.load("./conv.pt"))
conv.eval()
# count_in =0
# count = 0
# for i,ind in enumerate(range((len(flights)-1000),len(flights))):
#     if((i+1)%10000 == 0):
#         print(i)
#     X = flights.loc[ind-spequence_length:ind-1,:'DAY_OF_WEEK']
#     x = {'DAY_OF_WEEK':torch.tensor(X.DAY_OF_WEEK.to_list(),dtype=torch.float).reshape(1,spequence_length,-1),
#         'AIRLINE': torch.tensor(X.AIRLINE.to_list(),dtype=torch.long),
#          'ARRIVAL_DELAY': torch.tensor(X.ARRIVAL_DELAY.to_list(),dtype=torch.float),
#         'ORGIN_AIRPORT_V': torch.tensor(X.ORGIN_AIRPORT_V.to_list(),dtype=torch.long),
#         'DESTINATION__AIRPORT_V': torch.tensor(X.DESTINATION__AIRPORT_V.to_list(),dtype=torch.long)}
#     d = flights.loc[ind,'AIRLINE']
#     test_decode = torch.tensor([d],dtype=torch.long)
#     decode = decode.to(device)
#     out = conv(x,decode)
#     if(abs(out) <= 10):
#         count_in +=1
#     count+=1
for decode,y,x in validation_generator:
#             decode = decode.to(device)
            out = conv(x,decode)
            out = out
#             for i in out:
#                 if(abs(i) <= 10):
#                     count_in +=1
#                 count+=1
            print('The predicted value is {} and the actual {}'.format(out,y))

The predicted value is tensor([[4.9180],
        [8.5173]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 10.],
        [-19.]])
The predicted value is tensor([[7.7730],
        [4.3834]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -9.],
        [-19.]])
The predicted value is tensor([[9.2106],
        [8.7129]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-25.],
        [ 12.]])
The predicted value is tensor([[4.5551],
        [4.7093]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 5.],
        [18.]])
The predicted value is tensor([[12.2462],
        [18.5063]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-12.],
        [-10.]])
The predicted value is tensor([[26.3063],
        [ 5.9617]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[18.],
        [ 0.]])
The predicted value is tensor([[32.7992],
        [29.0917]], device='cuda:0', grad_fn=<AddmmBackwar

The predicted value is tensor([[ 5.1453],
        [10.1235]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-24.],
        [ 85.]])
The predicted value is tensor([[4.9974],
        [3.7227]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-14.],
        [  4.]])
The predicted value is tensor([[7.1966],
        [9.8420]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -1.],
        [-13.]])
The predicted value is tensor([[5.6301],
        [8.3911]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-31.],
        [ 21.]])
The predicted value is tensor([[8.8676],
        [8.7908]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-15.],
        [  8.]])
The predicted value is tensor([[8.4256],
        [1.3840]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -2.],
        [-19.]])
The predicted value is tensor([[2.6102],
        [7.6471]], device='cuda:0', grad_fn=<AddmmBackwar

The predicted value is tensor([[3.5843],
        [3.8015]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-13.],
        [-10.]])
The predicted value is tensor([[4.9052],
        [6.1879]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-20.],
        [ -1.]])
The predicted value is tensor([[8.3231],
        [7.0290]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-10.],
        [ -6.]])
The predicted value is tensor([[7.8178],
        [9.3662]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-19.],
        [ -9.]])
The predicted value is tensor([[9.6177],
        [1.6518]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-3.],
        [17.]])
The predicted value is tensor([[4.1429],
        [6.3804]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-11.],
        [-20.]])
The predicted value is tensor([[5.9070],
        [8.0675]], device='cuda:0', grad_fn=<AddmmBackward>) 

The predicted value is tensor([[5.1096],
        [6.7355]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[0.],
        [3.]])
The predicted value is tensor([[7.9277],
        [8.4982]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 10.],
        [-29.]])
The predicted value is tensor([[9.4588],
        [2.9438]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-9.],
        [44.]])
The predicted value is tensor([[3.5799],
        [5.3875]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 0.],
        [-4.]])
The predicted value is tensor([[5.5883],
        [7.1681]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-10.],
        [ 93.]])
The predicted value is tensor([[9.3100],
        [5.6536]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[3.],
        [3.]])
The predicted value is tensor([[7.4875],
        [5.1130]], device='cuda:0', grad_fn=<AddmmBackward>) and the ac

The predicted value is tensor([[3.4952],
        [2.5255]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[21.],
        [-1.]])
The predicted value is tensor([[6.1545],
        [8.9057]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-1.],
        [-4.]])
The predicted value is tensor([[2.5019],
        [3.1407]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-17.],
        [-14.]])
The predicted value is tensor([[6.1622],
        [8.7546]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 0.],
        [11.]])
The predicted value is tensor([[4.7368],
        [4.9877]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -4.],
        [-14.]])
The predicted value is tensor([[ 4.1355],
        [15.0985]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -4.],
        [-11.]])
The predicted value is tensor([[10.7514],
        [11.3976]], device='cuda:0', grad_fn=<AddmmBackward>) 

...

The predicted value is tensor([[7.4571],
        [8.6887]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[12.],
        [ 8.]])
The predicted value is tensor([[10.1490],
        [ 5.2297]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-6.],
        [24.]])
The predicted value is tensor([[6.8277],
        [7.8963]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-10.],
        [-17.]])
The predicted value is tensor([[2.6247],
        [3.8540]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[  2.],
        [-15.]])
The predicted value is tensor([[6.9299],
        [7.7977]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-7.],
        [-7.]])
The predicted value is tensor([[8.8685],
        [9.2783]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-4.],
        [-7.]])
The predicted value is tensor([[10.4479],
        [ 8.9415]], device='cuda:0', grad_fn=<AddmmBackward>) an

The predicted value is tensor([[4.2443],
        [3.1373]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-29.],
        [-30.]])
The predicted value is tensor([[3.2237],
        [5.1169]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-13.],
        [ -7.]])
The predicted value is tensor([[2.0924],
        [3.0636]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-4.],
        [-6.]])
The predicted value is tensor([[5.1738],
        [5.5986]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-26.],
        [-19.]])
The predicted value is tensor([[7.9685],
        [7.5935]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 5.],
        [-3.]])
The predicted value is tensor([[7.0103],
        [9.2891]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-6.],
        [-4.]])
The predicted value is tensor([[9.7289],
        [4.1058]], device='cuda:0', grad_fn=<AddmmBackward>) and 

The predicted value is tensor([[-1.0291],
        [-0.4339]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-6.],
        [-6.]])
The predicted value is tensor([[2.3925],
        [1.6308]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-35.],
        [ 29.]])
The predicted value is tensor([[ 0.4252],
        [-1.0046]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 11.],
        [-16.]])
The predicted value is tensor([[3.3826],
        [3.0955]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-11.],
        [-20.]])
The predicted value is tensor([[ 3.2866],
        [-0.5711]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-18.],
        [ -7.]])
The predicted value is tensor([[0.2960],
        [1.0575]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[  6.],
        [-10.]])
The predicted value is tensor([[-0.1529],
        [ 0.3574]], device='cuda:0', grad_fn=<AddmmBac

The predicted value is tensor([[4.8443],
        [7.3843]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-2.],
        [ 7.]])
The predicted value is tensor([[6.4354],
        [7.8140]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -4.],
        [-15.]])
The predicted value is tensor([[8.8676],
        [3.9660]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-19.],
        [ 58.]])
The predicted value is tensor([[2.4655],
        [4.3559]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[2.],
        [5.]])
The predicted value is tensor([[4.8153],
        [3.6488]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[125.],
        [ -2.]])
The predicted value is tensor([[4.7216],
        [8.5468]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-11.],
        [ -6.]])
The predicted value is tensor([[10.2703],
        [ 3.2343]], device='cuda:0', grad_fn=<AddmmBackward>) an

The predicted value is tensor([[3.5486],
        [7.2696]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ -4.],
        [-22.]])
The predicted value is tensor([[8.7014],
        [4.3247]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[11.],
        [ 6.]])
The predicted value is tensor([[5.8274],
        [8.2407]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[ 30.],
        [-11.]])
The predicted value is tensor([[10.6029],
        [ 4.2750]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[66.],
        [ 3.]])
The predicted value is tensor([[14.7949],
        [ 3.8622]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-7.],
        [13.]])
The predicted value is tensor([[5.3575],
        [7.8780]], device='cuda:0', grad_fn=<AddmmBackward>) and the actual tensor([[-21.],
        [ -2.]])
The predicted value is tensor([[7.9982],
        [9.7984]], device='cuda:0', grad_fn=<AddmmBackward>) 

In [22]:
# (count_in/count)*100

In [23]:
for i,ind in enumerate(range(len(flights)-100,len(flights))):
    if((i+1)%10 == 0):
        print(i)
    X = flights.loc[ind-spequence_length:ind-1,:'DAY_OF_WEEK']
    x = {'DAY_OF_WEEK':torch.tensor(X.DAY_OF_WEEK.to_list(),dtype=torch.float).reshape(1,spequence_length,-1),
        'AIRLINE': torch.tensor(X.AIRLINE.to_list(),dtype=torch.long),
         'ARRIVAL_DELAY': torch.tensor(X.ARRIVAL_DELAY.to_list(),dtype=torch.float),
        'ORGIN_AIRPORT_V': torch.tensor(X.ORGIN_AIRPORT_V.to_list(),dtype=torch.long),
        'DESTINATION__AIRPORT_V': torch.tensor(X.DESTINATION__AIRPORT_V.to_list(),dtype=torch.long)}
    d = flights.loc[ind,'AIRLINE']
    test_decode = torch.tensor([d],dtype=torch.long)
    decode = decode.to(device)
    out = conv(x,decode)
    a = flights.loc[ind,'ARRIVAL_DELAY']
    print('The predicted value is {} and the actual {}'.format(out,a))

AttributeError: 'dict' object has no attribute 'to'